In [1]:
from copy import deepcopy
from itertools import product

import numpy as np
import ocelot
from ocelot.cpbd.beam import generate_parray

/opt/homebrew/Caskroom/miniforge/base/envs/cheetah-demos/lib/python3.9/site-packages/ocelot/cpbd/beam.py:1567: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  slice_analysis = slice_analysis_py if not nb_flag else nb.jit(slice_analysis_py)
/opt/homebrew/Caskroom/miniforge/base/envs/cheetah-demos/lib/python3.9/site-packages/ocelot/cpbd/tm_utils.py:153: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behavi

initializing ocelot...
import: module PYFFTW is not installed. Install it to speed up calculation


/opt/homebrew/Caskroom/miniforge/base/envs/cheetah-demos/lib/python3.9/site-packages/ocelot/cpbd/wake3D.py:80: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  project_on_grid = project_on_grid_py if not nb_flag else nb.jit(project_on_grid_py)


In [2]:
quadrupole = ocelot.XYQuadrupole(
    l=np.random.uniform(0.05, 0.5),  # 0.2
    k1=np.random.uniform(-72.0, 72.0),  # 4.2
    x_offs=np.random.uniform(-0.5e-3, 0.5e-3),
    y_offs=np.random.uniform(-0.5e-3, 0.5e-3),
    tilt=np.random.uniform(-0.1, 0.1),  # 0.0
)
cell = [quadrupole]
cell

[<XYQuadrupole: name=ID_54984144_ at 0x1762d66d0>]

In [3]:
p_array_incoming = generate_parray(
    sigma_x=np.random.uniform(1e-5, 1e-3),  # 1e-4
    sigma_px=np.random.uniform(1e-5, 1e-3),  # 2e-5
    sigma_y=np.random.uniform(1e-5, 1e-3),  # None
    sigma_py=np.random.uniform(1e-5, 1e-3),  # None
    sigma_tau=np.random.uniform(1e-4, 1e-2),  # 1e-3
    sigma_p=np.random.uniform(1e-5, 1e-3),  # 1e-4
    chirp=np.random.uniform(-0.1, 0.1),  # 0.01
    charge=np.random.uniform(1e-9, 1e-7),  # 5e-9
    nparticles=np.random.randint(100_000, 1_000_000),  # 200_000
    energy=np.random.uniform(0.001, 15.0),  # 0.13 (I think this is in GeV)
)
p_array_incoming

In [4]:
method = {"global": ocelot.SecondTM}
lattice = ocelot.MagneticLattice(cell, method=method)

space_charge = ocelot.SpaceCharge()
space_charge.nmesh_xyz = [63, 63, 63]
space_charge.step = 1

navigator = ocelot.Navigator(lattice)
navigator.add_physics_proc(space_charge, quadrupole, quadrupole)
navigator.unit_step = 0.02

p_array = deepcopy(p_array_incoming)
_, p_array_outgoing = ocelot.track(lattice, p_array, navigator)

p_array_outgoing

z = 0.4460377934499288 / 0.4460377934499288. Applied: 

In [5]:
ocelot.save_particle_array("p_array_outgoing.npz", p_array_outgoing)